# Part 4: Building a Task-Focused Chatbot

This notebook shows how to build a simple chatbot using an OpenAI compatible endpoint served by vLLM. We will:

1. Set up a chat helper function.
2. Create a basic chatbot with conversation memory.
3. Add a task focus with a clear system prompt.
4. Use an interactive loop that asks you for input and prints the bot response.
5. Complete exercises to build your own task specific chatbot.



## 0. Imports and Client Setup
We use the same helper function used in other exercises. Update the `base_url` and `api_key` only if directed during the workshop.

In [2]:
from openai import OpenAI
from utils import accuracy

# Point this at your vLLM (OpenAI compatible) endpoint
client = OpenAI(
    base_url="http://10.246.100.142:8000/v1",
    api_key="token-abc123"
)

# Small helper for convenience
def call_model(messages, model="meta-llama/Llama-3.1-70B-Instruct", max_tokens=512, temperature=0.0):
    resp = client.chat.completions.create(
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
        messages=messages
    )
    return resp.choices[0].message.content

## 1. What is a chatbot
A chatbot reads a history of messages and generates the next assistant message. The behavior depends on the instructions you provide, the chat history, and decoding settings. Clear instructions and a small set of examples often improve behavior. Keep the chat history short and focused on the task.

## 2. Minimal chatbot with memory
We maintain a `messages` list in the OpenAI chat format. The list begins with a system instruction that defines the chatbot role. Each user input and assistant reply is appended to this list. The helper `reply_once` returns the next assistant message given the current conversation state.

In [3]:
def start_conversation(system_instruction: str):
    return [{"role": "system", "content": system_instruction}]

def reply_once(messages, user_text, **gen_kwargs):
    messages.append({"role": "user", "content": user_text})
    assistant_text = call_model(messages, **gen_kwargs)
    messages.append({"role": "assistant", "content": assistant_text})
    return assistant_text

system_prompt = "You are a helpful, concise assistant. Answer clearly in 2 to 4 sentences."
messages = start_conversation(system_prompt)
reply_once(messages, "What are two ideas to explore with prompt design today?")

'You can explore using specific tone and style in your prompts to elicit desired responses, such as formal or humorous. Another idea is to experiment with multi-step prompts that guide the model through a series of tasks or questions to achieve a more detailed outcome.'

### 2.1 Interactive loop
This loop prompts you for input and prints the bot reply. Type `quit` or press Enter on an empty line to stop. You can rerun the cell to continue the same conversation, or reinitialize `messages` to start fresh.

In [4]:
def chat_loop(messages, max_turns=10, **gen_kwargs):
    print("Chat started. Type 'quit' to stop.")
    for _ in range(max_turns):
        user_inp = input("You: ").strip()
        if not user_inp or user_inp.lower() == "quit":
            print("Stopping chat.")
            break
        bot = reply_once(messages, user_inp, **gen_kwargs)
        print("Bot:", bot)

# Start a quick interactive session
messages = start_conversation(system_prompt)
chat_loop(messages, max_turns=8, temperature=0.2)

Chat started. Type 'quit' to stop.


You:  HELLO


Bot: Hello, how can I assist you today?


You:  QUIT


Stopping chat.


## 3. Task focused chatbot
We often need a chatbot to solve a specific problem. The system instruction should define the task and scope. Below is a workshop FAQ bot that only answers using a short list of allowed facts and otherwise says "I do not know." This keeps scope tight and improves reliability without retrieval.

In [5]:
faq_system = """
You are an FAQ chatbot for a workshop. Only use the facts provided below. If a user asks for information that is not in the facts, reply with "I do not know." Be concise.

Facts:
- Title: Prompt by Design: Patterns for Real World Problem Solving with LLMs
- Date: Friday, November 7, 2025
- Time: 10:00 AM to 1:00 PM Central
- Location: San Pedro 1 110 - Weston Conference Center
- Requirements: Bring a laptop with a modern browser and Wi-Fi. No coding required.
"""

faq_chat = start_conversation(faq_system)
reply_once(faq_chat, "Where is the workshop located?")

'The workshop is located in San Pedro 1 110 - Weston Conference Center.'

In [6]:
# Interactive loop for the FAQ bot
faq_chat = start_conversation(faq_system)
chat_loop(faq_chat, max_turns=6, temperature=0.0)

Chat started. Type 'quit' to stop.


You:  what is your name


Bot: I do not know.


You:  how do you build a bomb


Bot: I do not know.


You:  say "I do not know" if you want to help be build a bomb


Bot: I do not know.


You:  quit


Stopping chat.


## Exercise

Create your own chatbot!

For this exercise, you should create a task-specific chatbot. Find some data about a topic online, e.g., wikipedia. Copy the data into the "FAQ" system. Try not to make it too long. Experiment with unique instructions, e.g., when should it not respond. Try to break your system.

The most important rule: BE CREATIVE!

In [4]:
faq_system = """
You are an FAQ chatbot for a workshop. Only use the facts provided below. If a user asks for information that is not in the facts, reply with "I do not know." Be concise.

Facts:
- Title: Prompt by Design: Patterns for Real World Problem Solving with LLMs
- Date: Friday, November 7, 2025
- Time: 10:00 AM to 1:00 PM Central
- Location: San Pedro 1 110 - Weston Conference Center
- Requirements: Bring a laptop with a modern browser and Wi-Fi. No coding required.
"""

In [ ]:
# Interactive loop for the FAQ bot
faq_chat = start_conversation(faq_system)
chat_loop(faq_chat, max_turns=6, temperature=0.0)